## Test Data

The aim of this project is to create and evaluate regression model for predicting the price of a house in Ames city at the time of sale. 

This notebook contains data cleaning and processing of test data before predicting sale price.

### Contents:
- [Data Import](#Data-Import)
- [Renaming Columns](#Renaming-Columns)
- [Data Cleaning](#Data-Cleaning)
    - [1. Imputing Null Values](#1.-Imputing-Null-Values)
    - [2. Checking Datatypes](#2.-Checking-Datatypes)
    - [3. Handling Ordinal Fields](#3.-Handling-Ordinal-Fields)
    - [4. Manipulating Fields](#4.-Manipulating-Fields)
    - [5. Dropping Columns](#5.-dropping-Columns)
-[One Hot Encoding](#One-Hot-Encoding)

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

## Data Import

In [87]:
test = pd.read_csv('../datasets/test.csv')

In [88]:
pd.set_option('display.max_columns', len(test.columns))
pd.set_option('display.max_rows', len(test))

In [89]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [90]:
test.shape

(879, 80)

## Renaming Columns

In [91]:
test.columns = test.columns.map(lambda x: x.lower().replace(' ', '_'))
print(test.columns)

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

As we already have a unique column named id, we can use it as index.

In [92]:
test.set_index('id', inplace = True)

## Data Cleaning

### 1) Imputing Null Values

In [93]:
test.isnull().sum()

pid                  0
ms_subclass          0
ms_zoning            0
lot_frontage       160
lot_area             0
street               0
alley              821
lot_shape            0
land_contour         0
utilities            0
lot_config           0
land_slope           0
neighborhood         0
condition_1          0
condition_2          0
bldg_type            0
house_style          0
overall_qual         0
overall_cond         0
year_built           0
year_remod/add       0
roof_style           0
roof_matl            0
exterior_1st         0
exterior_2nd         0
mas_vnr_type         1
mas_vnr_area         1
exter_qual           0
exter_cond           0
foundation           0
bsmt_qual           25
bsmt_cond           25
bsmt_exposure       25
bsmtfin_type_1      25
bsmtfin_sf_1         0
bsmtfin_type_2      25
bsmtfin_sf_2         0
bsmt_unf_sf          0
total_bsmt_sf        0
heating              0
heating_qc           0
central_air          0
electrical           1
1st_flr_sf 

We can see that there are null values for several columns that we will need to address first.

There is one record which has garage_yr_blt, garage_finish, garage_qual and garage_cond as null, but there is value present in garage_type. We will replace them with maximum occuring value.

In [94]:
test.loc[(test['garage_yr_blt'].isnull()) & (test['garage_type'].notnull()), 'garage_yr_blt'] = test['garage_yr_blt'].median()
test.loc[(test['garage_finish'].isnull()) & (test['garage_type'].notnull()), 'garage_finish'] = test['garage_finish'].mode()
test.loc[(test['garage_qual'].isnull()) & (test['garage_type'].notnull()), 'garage_qual'] = test['garage_qual'].mode()
test.loc[(test['garage_cond'].isnull()) & (test['garage_type'].notnull()), 'garage_cond'] = test['garage_cond'].mode()

Rest all the null values represent the absence of the feature hence we will set them as NA or 0 , according to their respective types as we performed the same step for train data.

In [95]:
test.fillna(value={
    'lot_frontage' : test['lot_frontage'].median(),
    'alley' : 'NA',
    'mas_vnr_type' : 'NA',
    'mas_vnr_area' : 0,
    'bsmt_qual' : 'NA',
    'bsmt_cond' : 'NA',
    'bsmt_exposure' : 'NA',
    'bsmtfin_type_1' : 'NA',
    'bsmtfin_type_2' : 'NA',
    'fireplace_qu' : 'NA', 
    'garage_type' : 'NA',
    'garage_finish' : 'NA',
    'garage_yr_blt' : 0,
    'garage_qual' : 'NA',
    'garage_cond' : 'NA',
    'electrical' : 'SBrKr',
    'pool_qc' : 'NA',
    'fence' : 'NA',
    'misc_feature' : 'NA'
}, inplace = True)

Confirm that there are no more null values

In [96]:
test.isnull().sum()

pid                0
ms_subclass        0
ms_zoning          0
lot_frontage       0
lot_area           0
street             0
alley              0
lot_shape          0
land_contour       0
utilities          0
lot_config         0
land_slope         0
neighborhood       0
condition_1        0
condition_2        0
bldg_type          0
house_style        0
overall_qual       0
overall_cond       0
year_built         0
year_remod/add     0
roof_style         0
roof_matl          0
exterior_1st       0
exterior_2nd       0
mas_vnr_type       0
mas_vnr_area       0
exter_qual         0
exter_cond         0
foundation         0
bsmt_qual          0
bsmt_cond          0
bsmt_exposure      0
bsmtfin_type_1     0
bsmtfin_sf_1       0
bsmtfin_type_2     0
bsmtfin_sf_2       0
bsmt_unf_sf        0
total_bsmt_sf      0
heating            0
heating_qc         0
central_air        0
electrical         0
1st_flr_sf         0
2nd_flr_sf         0
low_qual_fin_sf    0
gr_liv_area        0
bsmt_full_bat

### 2) Checking Data Types

In [97]:
test.dtypes

pid                  int64
ms_subclass          int64
ms_zoning           object
lot_frontage       float64
lot_area             int64
street              object
alley               object
lot_shape           object
land_contour        object
utilities           object
lot_config          object
land_slope          object
neighborhood        object
condition_1         object
condition_2         object
bldg_type           object
house_style         object
overall_qual         int64
overall_cond         int64
year_built           int64
year_remod/add       int64
roof_style          object
roof_matl           object
exterior_1st        object
exterior_2nd        object
mas_vnr_type        object
mas_vnr_area       float64
exter_qual          object
exter_cond          object
foundation          object
bsmt_qual           object
bsmt_cond           object
bsmt_exposure       object
bsmtfin_type_1      object
bsmtfin_sf_1         int64
bsmtfin_type_2      object
bsmtfin_sf_2         int64
b

Pid and MS Subclass are nominal fields, so cannot keep it in numeric form. Converting it to string.

In [98]:
test['ms_subclass'] = test['ms_subclass'].astype('str')
test['pid'] = test['pid'].astype('str')

### 3) Handling Ordinal Fields

Now if we look at the data, there are 23 ordinal fields. Ordinal fields have values like Excellent, Good, Average, etc. We will  convert them to numeric ranking so we can use it for modelling data.  

In [99]:
test.replace({
    'lot_shape' : {'Reg':4, 'IR1':3, 'IR2':2, 'IR3':1},
    'utilities' : {'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1},
    'land_slope' : {'Gtl':4, 'Mod':3, 'Sev':2},
    'exter_qual' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1},
    'exter_cond' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1},
    'bsmt_qual' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1 , 'NA': 0},
    'bsmt_cond' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1 , 'NA': 0},
    'bsmt_exposure' : {'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'NA':0},
    'bsmtfin_type_1' : {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1 , 'NA':0},
    'bsmtfin_type_2' : {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1 , 'NA':0},
    'heating_qc' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1},
    'electrical' : {'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix':1},
    'kitchen_qual' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1},
    'functional' : {'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2': 2, 'Sev':1 , 'Sal':0},
    'fireplace_qu' : {'Ex':5, 'Gd':4, 'TA':3,'Fa':2, 'Po':1, 'NA':0},
    'garage_finish' : {'Fin':3, 'RFn':2, 'Unf':1, 'NA':0},
    'garage_qual' : {'Ex':5, 'Gd':4, 'TA':3,'Fa':2, 'Po':1, 'NA':0},
    'garage_cond' : {'Ex':5, 'Gd':4, 'TA':3,'Fa':2, 'Po':1, 'NA':0},
    'paved_drive' : {'Y':3, 'P':2, 'N':1},
    'pool_qc' : {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'NA': 0},
    'fence' : {'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'NA':0}
}, inplace = True)

In [100]:
test.head()

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,4,Lvl,4,Inside,4,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,2,Stone,2,3,1,1,0,1,0,1020,1020,GasA,4,N,2,908,1020,0,1928,0,0,2,0,4,2,2,9,7,0,0,Detchd,1910.0,1,1,440,1,1,3,0,60,112,0,0,0,0,0,NA,0,4,2006,WD
2718,905108090,90,RL,68.0,9662,Pave,NA,3,Lvl,4,Inside,4,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,3,3,CBlock,4,3,1,1,0,1,0,1967,1967,GasA,3,Y,5,1967,0,0,1967,0,0,2,0,6,2,3,10,7,0,0,Attchd,1977.0,3,2,580,3,3,3,170,0,0,0,0,0,0,0,NA,0,8,2006,WD
2414,528218130,60,RL,58.0,17104,Pave,NA,3,Lvl,4,Inside,4,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,3,PConc,4,4,3,6,554,1,0,100,654,GasA,5,Y,5,664,832,0,1496,1,0,2,1,3,1,4,7,7,1,4,Attchd,2006.0,2,2,426,3,3,3,100,24,0,0,0,0,0,0,NA,0,9,2006,New
1989,902207150,30,RM,60.0,8520,Pave,NA,4,Lvl,4,Inside,4,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,3,CBlock,3,3,1,1,0,1,0,968,968,GasA,3,Y,5,968,0,0,968,0,0,1,0,2,1,3,5,7,0,0,Detchd,1935.0,1,2,480,2,3,1,0,0,184,0,0,0,0,0,NA,0,7,2007,WD
625,535105100,20,RL,68.0,9500,Pave,NA,3,Lvl,4,Inside,4,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,3,3,CBlock,4,3,1,4,609,1,0,785,1394,GasA,4,Y,5,1394,0,0,1394,1,0,1,1,3,1,3,6,7,2,4,Attchd,1963.0,2,2,514,3,3,3,0,76,0,0,185,0,0,0,NA,0,7,2009,WD


In [101]:
nominal_columns = test.select_dtypes(include = ['object']).columns.to_list()
nominal_columns.remove('pid')

### 4) Manipulating Fields

Adding house_age field using year_built as done for train data

In [102]:
now = datetime.datetime.now()

In [103]:
test['year_built'] = now.year - test['year_built']
test.rename(columns={'year_built': 'house_age'}, inplace = True)

### 5) Dropping Columns

pid is Parcel identification number which can be used with city web site for parcel review.
As it is not a potential feature for predicting Sale Price we will drop it as we did for train.

In [104]:
test.drop(['pid'], axis = 1, inplace = True)

Furthermore, we will also drop all the fields that we dropped from train data after performing the EDA

In [105]:
test.drop('utilities', axis = 1, inplace = True)
test.drop('bsmtfin_sf_2', axis = 1, inplace = True)
test.drop('low_qual_fin_sf', axis = 1, inplace = True)
test.drop('misc_val', axis = 1, inplace = True)
test.drop('3ssn_porch', axis = 1, inplace = True)

### One Hot Encoding

Before we start modelling it is very important to convert the Nominal fields to dummies (0 & 1) as we cannot provide text values.

In [106]:
test.head()

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,house_age,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,mo_sold,yr_sold,sale_type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,190,RM,69.0,9142,Pave,Grvl,4,Lvl,Inside,4,OldTown,Norm,Norm,2fmCon,2Story,6,8,111,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,3,2,Stone,2,3,1,1,0,1,1020,1020,GasA,4,N,2,908,1020,1928,0,0,2,0,4,2,2,9,7,0,0,Detchd,1910.0,1,1,440,1,1,3,0,60,112,0,0,0,0,NA,4,2006,WD
2718,90,RL,68.0,9662,Pave,NA,3,Lvl,Inside,4,Sawyer,Norm,Norm,Duplex,1Story,5,4,44,1977,Gable,CompShg,Plywood,Plywood,None,0.0,3,3,CBlock,4,3,1,1,0,1,1967,1967,GasA,3,Y,5,1967,0,1967,0,0,2,0,6,2,3,10,7,0,0,Attchd,1977.0,3,2,580,3,3,3,170,0,0,0,0,0,0,NA,8,2006,WD
2414,60,RL,58.0,17104,Pave,NA,3,Lvl,Inside,4,Gilbert,Norm,Norm,1Fam,2Story,7,5,15,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,3,PConc,4,4,3,6,554,1,100,654,GasA,5,Y,5,664,832,1496,1,0,2,1,3,1,4,7,7,1,4,Attchd,2006.0,2,2,426,3,3,3,100,24,0,0,0,0,0,NA,9,2006,New
1989,30,RM,60.0,8520,Pave,NA,4,Lvl,Inside,4,OldTown,Norm,Norm,1Fam,1Story,5,6,98,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,4,3,CBlock,3,3,1,1,0,1,968,968,GasA,3,Y,5,968,0,968,0,0,1,0,2,1,3,5,7,0,0,Detchd,1935.0,1,2,480,2,3,1,0,0,184,0,0,0,0,NA,7,2007,WD
625,20,RL,68.0,9500,Pave,NA,3,Lvl,Inside,4,NAmes,Norm,Norm,1Fam,1Story,6,5,58,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,3,3,CBlock,4,3,1,4,609,1,785,1394,GasA,4,Y,5,1394,0,1394,1,0,1,1,3,1,3,6,7,2,4,Attchd,1963.0,2,2,514,3,3,3,0,76,0,185,0,0,0,NA,7,2009,WD


In [107]:
test.shape

(879, 73)

In [108]:
def encode_concat(df, columns_to_encode):
    '''
    This method creates dummy columns for all nominal fields, 
    concat it to the original dataframe and drop the original nominal columns.
    '''
    dummies = pd.get_dummies(df[columns_to_encode])
    dummies_nona = [d for d in dummies.columns if not 'NA' in d]
    df_concat = pd.concat([df, dummies[dummies_nona]], axis = 1)
    df_concat.drop(columns_to_encode, axis = 1, inplace = True)
    return df_concat

In [109]:
test_updated = encode_concat(test, nominal_columns)

In [110]:
test_updated.head()

,lot_frontage,lot_area,lot_shape,land_slope,overall_qual,overall_cond,house_age,year_remod/add,mas_vnr_area,exter_qual,exter_cond,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmt_unf_sf,total_bsmt_sf,heating_qc,1st_flr_sf,2nd_flr_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,...,mas_vnr_type_BrkFace,mas_vnr_type_CBlock,mas_vnr_type_None,mas_vnr_type_Stone,foundation_BrkTil,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood,heating_Floor,heating_GasA,heating_GasW,heating_Grav,central_air_N,central_air_Y,electrical_2,electrical_3,electrical_4,electrical_5,electrical_SBrKr,garage_type_2Types,garage_type_Attchd,garage_type_Basment,garage_type_BuiltIn,garage_type_CarPort,garage_type_Detchd,misc_feature_Gar2,misc_feature_Othr,misc_feature_Shed,sale_type_COD,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2658,69.0,9142,4,4,6,8,111,1950,0.0,3,2,2,3,1,1,0,1,1020,1020,4,908,1020,1928,0,0,2,0,4,2,2,9,7,0,0,1910.0,1,1,440,1,1,...,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2718,68.0,9662,3,4,5,4,44,1977,0.0,3,3,4,3,1,1,0,1,1967,1967,3,1967,0,1967,0,0,2,0,6,2,3,10,7,0,0,1977.0,3,2,580,3,3,...,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2414,58.0,17104,3,4,7,5,15,2006,0.0,4,3,4,4,3,6,554,1,100,654,5,664,832,1496,1,0,2,1,3,1,4,7,7,1,4,2006.0,2,2,426,3,3,...,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1989,60.0,8520,4,4,5,6,98,2006,0.0,4,3,3,3,1,1,0,1,968,968,3,968,0,968,0,0,1,0,2,1,3,5,7,0,0,1935.0,1,2,480,2,3,...,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
625,68.0,9500,3,4,6,5,58,1963,247.0,3,3,4,3,1,4,609,1,785,1394,4,1394,0,1394,1,0,1,1,3,1,3,6,7,2,4,1963.0,2,2,514,3,3,...,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [111]:
test_updated.shape

(879, 217)

Exporting the updated test data to csv file for further model evaluation.

In [112]:
test_updated.to_csv('../datasets/housing_test_cleaned.csv')